# VDEH Data Enrichment - Library of Congress (LoC)

**Fokus:** Datenanreicherung englischsprachiger Literatur über Library of Congress (LoC) API

## 🎯 Ziel
- Anreicherung englischsprachiger Datensätze via LoC API (ISBN/ISSN)
- Ergänzung zu DNB-Daten für internationale Literatur
- Fokus auf Records mit `detected_language = 'en'`
- Validierung und Qualitätsverbesserung

## 📚 Input/Output
- **Input**: `data/vdeh/processed/03_language_detected_data.parquet`
- **Output**: `data/vdeh/processed/04b_loc_enriched_data.parquet`

## 🔗 API
- **LoC SRU API**: https://www.loc.gov/z3950/
- **Endpoint**: https://lx2.loc.gov:210/lcdb
- **Abfrage**: ISBN/ISSN basierte Suche, Titel/Autor


In [1]:
# 🛠️ SETUP UND DATEN LADEN
import sys
from pathlib import Path
import time
import pandas as pd
import json

# Add project root to path
project_root = Path.cwd()
while not (project_root / 'config.yaml').exists() and project_root.parent != project_root:
    project_root = project_root.parent
sys.path.insert(0, str(project_root / 'src'))

from utils.notebook_utils import setup_notebook

project_root, config = setup_notebook()
print(f"✅ Project root: {project_root}")
print(f"✅ Project: {config.get('project.name')} v{config.get('project.version')}")

# LoC API laden
from loc_api import LOC_SRU_BASE, query_loc_by_isbn, query_loc_by_issn, query_loc_by_title_author
print(f"✅ LoC API Funktionen geladen")

2025-12-25 21:57:58 - utils.notebook_utils - INFO - Searching for project root...


2025-12-25 21:57:58 - utils.notebook_utils - INFO - Project root found: /media/sz/Data/Bibo/analysis


2025-12-25 21:57:58 - utils.notebook_utils - INFO - Loading configuration...


2025-12-25 21:57:58 - config_loader - INFO - Configuration loaded from /media/sz/Data/Bibo/analysis/config.yaml


2025-12-25 21:57:58 - utils.notebook_utils - INFO - Configuration loaded successfully: Dual-Source Bibliothek Bestandsvergleich


✅ Project root: /media/sz/Data/Bibo/analysis
✅ Project: Dual-Source Bibliothek Bestandsvergleich v2.2.0
✅ LoC API Funktionen geladen


In [2]:
# 📂 DATEN AUS VORHERIGER STUFE LADEN
processed_dir = config.project_root / config.get('paths.data.vdeh.processed')
input_path = processed_dir / '03_language_detected_data.parquet'
metadata_path = processed_dir / '03_metadata.json'

if not input_path.exists():
    raise FileNotFoundError(f"Input-Datei nicht gefunden: {input_path}\n"
                          "Bitte führen Sie zuerst 03_vdeh_language_detection.ipynb aus.")

# Daten laden
df_vdeh = pd.read_parquet(input_path)

# Vorherige Metadaten laden
with open(metadata_path, 'r') as f:
    prev_metadata = json.load(f)

print(f"📂 Daten geladen aus: {input_path}")
print(f"📊 Records: {len(df_vdeh):,}")
print(f"📋 Spalten: {list(df_vdeh.columns)}")
print(f"💾 Memory: {df_vdeh.memory_usage(deep=True).sum() / 1024**2:.1f} MB")

# Sprachverteilung
if 'detected_language' in df_vdeh.columns:
    lang_dist = df_vdeh['detected_language'].value_counts()
    print(f"\n📊 Sprachverteilung:")
    for lang, count in lang_dist.head(10).items():
        print(f"   {lang}: {count:,} ({count/len(df_vdeh)*100:.1f}%)")

📂 Daten geladen aus: /media/sz/Data/Bibo/analysis/data/vdeh/processed/03_language_detected_data.parquet
📊 Records: 58,305
📋 Spalten: ['id', 'title', 'authors', 'authors_affiliation', 'year', 'publisher', 'isbn', 'issn', 'pages', 'language', 'authors_str', 'num_authors', 'authors_affiliation_str', 'num_authors_affiliation', 'isbn_valid', 'isbn_status', 'issn_valid', 'issn_status', 'detected_language', 'detected_language_confidence', 'detected_language_name']


💾 Memory: 54.5 MB

📊 Sprachverteilung:
   de: 32,543 (55.8%)
   en: 16,516 (28.3%)
   unknown: 1,446 (2.5%)
   fr: 929 (1.6%)
   it: 773 (1.3%)
   sl: 654 (1.1%)
   ro: 625 (1.1%)
   af: 520 (0.9%)
   nl: 497 (0.9%)
   ca: 493 (0.8%)


In [3]:
# 🔍 KANDIDATEN FÜR LOC-ANREICHERUNG IDENTIFIZIEREN
print("🔍 === KANDIDATEN-IDENTIFIKATION (ENGLISCHSPRACHIGE LITERATUR) ===\n")

# Filter: Nur englischsprachige Records
df_english = df_vdeh[df_vdeh['detected_language'] == 'en'].copy()

print(f"📚 Englischsprachige Records: {len(df_english):,} ({len(df_english)/len(df_vdeh)*100:.1f}%)")

# 1. ISBN-basierte Kandidaten (ALLE mit ISBN)
if 'isbn' in df_english.columns:
    has_isbn = df_english['isbn'].notna()
    isbn_candidates = df_english[has_isbn].copy()
    
    print(f"\n📋 Kriterium 1: ISBN vorhanden")
    print(f"   ISBN-Kandidaten: {len(isbn_candidates):,}")

# 2. ISSN-basierte Kandidaten (nur unvollständig)
if 'issn' in df_english.columns:
    has_issn = df_english['issn'].notna()
    missing_title = df_english['title'].isna()
    missing_authors = (df_english['authors_str'].isna()) | (df_english['authors_str'] == '')
    missing_year = df_english['year'].isna()
    
    issn_candidates = df_english[
        has_issn & (missing_title | missing_authors | missing_year)
    ].copy()
    
    print(f"\n📋 Kriterium 2: ISSN vorhanden (nur unvollständig)")
    print(f"   ISSN-Kandidaten: {len(issn_candidates):,}")

# 3. Titel/Autor-Kandidaten (ohne ISBN aber mit Titel + Autoren)
no_isbn_but_searchable = df_english[
    (df_english['isbn'].isna()) &
    (df_english['title'].notna()) &
    (df_english['authors_str'].notna()) &
    (df_english['authors_str'] != '')
].copy()

print(f"\n📋 Kriterium 3: Ohne ISBN aber mit Titel + Autoren")
print(f"   Kandidaten: {len(no_isbn_but_searchable):,}")

# Finale Kandidatenliste
final_candidates = df_english[
    (df_english['isbn'].notna()) |
    (
        (df_english['isbn'].isna()) &
        (df_english['title'].notna()) &
        (
            ((df_english['authors_str'].notna()) & (df_english['authors_str'] != '')) |
            (df_english['year'].notna())
        )
    )
].copy()

print(f"\n🎯 Finale LoC-Anreicherungs-Kandidaten: {len(final_candidates):,}")
print(f"   Mit ISBN: {final_candidates['isbn'].notna().sum():,}")
print(f"   Mit ISSN: {final_candidates['issn'].notna().sum():,}")
print(f"   Mit Titel+Autoren: {((final_candidates['title'].notna()) & (final_candidates['authors_str'].notna()) & (final_candidates['authors_str'] != '')).sum():,}")


🔍 === KANDIDATEN-IDENTIFIKATION (ENGLISCHSPRACHIGE LITERATUR) ===

📚 Englischsprachige Records: 16,516 (28.3%)

📋 Kriterium 1: ISBN vorhanden
   ISBN-Kandidaten: 3,695

📋 Kriterium 2: ISSN vorhanden (nur unvollständig)
   ISSN-Kandidaten: 224

📋 Kriterium 3: Ohne ISBN aber mit Titel + Autoren
   Kandidaten: 2,112

🎯 Finale LoC-Anreicherungs-Kandidaten: 9,900
   Mit ISBN: 3,695
   Mit ISSN: 103
   Mit Titel+Autoren: 4,770


In [4]:
# 🌐 LOC API STATUS
print("🌐 === LIBRARY OF CONGRESS API STATUS ===\n")

print("✅ LoC API Funktionen aus src/loc_api.py geladen")
print(f"   Base URL: {LOC_SRU_BASE}")
print(f"   Schema: MARC21-xml")
print(f"   Verfügbare Funktionen:")
print(f"     - query_loc_by_isbn(isbn, max_records=1)")
print(f"     - query_loc_by_issn(issn, max_records=1)")
print(f"     - query_loc_by_title_author(title, author=None, max_records=1)")


🌐 === LIBRARY OF CONGRESS API STATUS ===

✅ LoC API Funktionen aus src/loc_api.py geladen
   Base URL: http://lx2.loc.gov:210/lcdb
   Schema: MARC21-xml
   Verfügbare Funktionen:
     - query_loc_by_isbn(isbn, max_records=1)
     - query_loc_by_issn(issn, max_records=1)
     - query_loc_by_title_author(title, author=None, max_records=1)


In [5]:
# 🚀 LOC DATENABFRAGE - ISBN/ISSN
print("🚀 === LOC DATENABFRAGE (ISBN/ISSN) ===\n")

# Konfiguration
RATE_LIMIT_DELAY = 10.0  # Sekunden zwischen Anfragen (erhöht wegen Verbindungsproblemen)
SAVE_INTERVAL = 50  # Speichere alle N Abfragen
LOC_DATA_FILE = processed_dir / 'loc_raw_data.parquet'

print(f"⚙️  Konfiguration:")
print(f"   Rate Limit: {RATE_LIMIT_DELAY}s pro Anfrage")
print(f"   Save Interval: Alle {SAVE_INTERVAL} Queries")
print(f"   Output: {LOC_DATA_FILE.name}")

# Lade vorhandene LoC-Daten (falls vorhanden)
if LOC_DATA_FILE.exists():
    print(f"\n📂 Lade vorhandene LoC-Daten...")
    loc_data_df = pd.read_parquet(LOC_DATA_FILE)
    print(f"   Bereits abgefragt: {len(loc_data_df):,}")
    print(f"   Davon erfolgreich: {(loc_data_df['loc_found'] == True).sum():,}")
else:
    print(f"\n📂 Keine vorhandenen LoC-Daten gefunden - starte neue Abfrage")
    loc_data_df = pd.DataFrame(columns=[
        'vdeh_id', 'query_type', 'query_value',
        'loc_found', 'loc_title', 'loc_authors', 'loc_year', 'loc_publisher',
        'loc_isbn', 'loc_issn', 'loc_pages'
    ])

# Sammle ISBN/ISSN aus englischsprachigen Kandidaten
print(f"\n📋 Extrahiere ISBN/ISSN aus {len(final_candidates):,} Kandidaten...")

queries_isbn = final_candidates[final_candidates['isbn'].notna()][['id', 'isbn']].copy()
queries_isbn.columns = ['vdeh_id', 'query_value']
queries_isbn['query_type'] = 'ISBN'

queries_issn = final_candidates[
    final_candidates['isbn'].isna() & final_candidates['issn'].notna()
][['id', 'issn']].copy()
queries_issn.columns = ['vdeh_id', 'query_value']
queries_issn['query_type'] = 'ISSN'

all_queries = pd.concat([queries_isbn, queries_issn], ignore_index=True)

print(f"   ISBN-Queries: {len(queries_isbn):,}")
print(f"   ISSN-Queries: {len(queries_issn):,}")
print(f"   Gesamt: {len(all_queries):,}")

# Filtere bereits abgefragte
if len(loc_data_df) > 0:
    already_queried = set(loc_data_df['query_value'])
    new_queries = all_queries[~all_queries['query_value'].isin(already_queried)].copy()
    
    print(f"\n🔍 Abgleich mit vorhandenen Daten:")
    print(f"   Bereits vorhanden: {len(all_queries) - len(new_queries):,}")
    print(f"   Neu abzufragen: {len(new_queries):,}")
else:
    new_queries = all_queries
    print(f"\n🔍 Alle {len(new_queries):,} Queries sind neu")

# Nur abfragen wenn neue Queries vorhanden
if len(new_queries) > 0:
    print(f"\n🔄 Starte LoC-Abfrage für {len(new_queries):,} neue Queries...\n")
    
    from tqdm.auto import tqdm
    
    results = []
    stats = {'found': 0, 'not_found': 0}
    query_count = 0
    
    for _, row in tqdm(new_queries.iterrows(), total=len(new_queries), desc="🔍 LoC API", unit="queries"):
        # API-Abfrage
        loc_result = None
        if row['query_type'] == 'ISBN':
            loc_result = query_loc_by_isbn(row['query_value'])
        elif row['query_type'] == 'ISSN':
            loc_result = query_loc_by_issn(row['query_value'])
        
        # Ergebnis speichern
        result_row = {
            'vdeh_id': row['vdeh_id'],
            'query_type': row['query_type'],
            'query_value': row['query_value'],
            'loc_found': loc_result is not None,
            'loc_title': loc_result.get('title') if loc_result else None,
            'loc_authors': ', '.join(loc_result.get('authors', [])) if loc_result else None,
            'loc_year': loc_result.get('year') if loc_result else None,
            'loc_publisher': loc_result.get('publisher') if loc_result else None,
            'loc_isbn': loc_result.get('isbn') if loc_result else None,
            'loc_issn': loc_result.get('issn') if loc_result else None,
            'loc_pages': loc_result.get('pages') if loc_result else None
        }
        
        results.append(result_row)
        
        if loc_result:
            stats['found'] += 1
        else:
            stats['not_found'] += 1
        
        query_count += 1
        
        # Regelmäßiges Speichern
        if query_count % SAVE_INTERVAL == 0:
            new_results_df = pd.DataFrame(results)
            loc_data_df = pd.concat([loc_data_df, new_results_df], ignore_index=True)
            loc_data_df.to_parquet(LOC_DATA_FILE, index=False)
            results = []
            print(f"💾 Zwischenspeicherung: {query_count}/{len(new_queries)} Queries abgefragt")
        
        # Rate Limiting
        time.sleep(RATE_LIMIT_DELAY)
    
    # Finale Speicherung
    if len(results) > 0:
        new_results_df = pd.DataFrame(results)
        loc_data_df = pd.concat([loc_data_df, new_results_df], ignore_index=True)
        loc_data_df.to_parquet(LOC_DATA_FILE, index=False)
    
    print(f"\n💾 LoC-Daten gespeichert: {LOC_DATA_FILE.name}")
    
    # Zusammenfassung
    print(f"\n📊 === NEUE ABFRAGEN ===")
    print(f"   Neue Queries: {len(new_queries):,}")
    print(f"   ✅ Gefunden: {stats['found']:,} ({stats['found']/len(new_queries)*100:.1f}%)")
    print(f"   ❌ Nicht gefunden: {stats['not_found']:,} ({stats['not_found']/len(new_queries)*100:.1f}%)")

else:
    print(f"\n✅ Alle ISBN/ISSN bereits in LoC-Daten vorhanden - keine neuen Abfragen nötig")

# Gesamtstatistik
print(f"\n📊 === GESAMT LOC-DATEN ===")
print(f"   Total Records: {len(loc_data_df):,}")
print(f"   Erfolgreich: {(loc_data_df['loc_found'] == True).sum():,}")
print(f"   Nicht gefunden: {(loc_data_df['loc_found'] == False).sum():,}")

🚀 === LOC DATENABFRAGE (ISBN/ISSN) ===

⚙️  Konfiguration:
   Rate Limit: 10.0s pro Anfrage
   Save Interval: Alle 50 Queries
   Output: loc_raw_data.parquet

📂 Lade vorhandene LoC-Daten...
   Bereits abgefragt: 3,785
   Davon erfolgreich: 797

📋 Extrahiere ISBN/ISSN aus 9,900 Kandidaten...
   ISBN-Queries: 3,695
   ISSN-Queries: 90
   Gesamt: 3,785

🔍 Abgleich mit vorhandenen Daten:
   Bereits vorhanden: 3,785
   Neu abzufragen: 0

✅ Alle ISBN/ISSN bereits in LoC-Daten vorhanden - keine neuen Abfragen nötig

📊 === GESAMT LOC-DATEN ===
   Total Records: 3,785
   Erfolgreich: 797
   Nicht gefunden: 2,988


In [6]:
# 🔍 LOC TITEL/AUTOR-SUCHE
print("🔍 === LOC TITEL/AUTOR-SUCHE ===\n")

LOC_TITLE_DATA_FILE = processed_dir / 'loc_title_author_data.parquet'
ALWAYS_TA_FOR_ALL_WITH_TITLE_AUTHORS = True

print(f"⚙️  Konfiguration:")
print(f"   Rate Limit: {RATE_LIMIT_DELAY}s pro Anfrage")
print(f"   Save Interval: Alle {SAVE_INTERVAL} Queries")
print(f"   Output: {LOC_TITLE_DATA_FILE.name}")
print(f"   TA für alle Titel+Autoren: {ALWAYS_TA_FOR_ALL_WITH_TITLE_AUTHORS}")

# Lade vorhandene Titel/Autor-Suchdaten
if LOC_TITLE_DATA_FILE.exists():
    print(f"\n📂 Lade vorhandene Titel/Autor-Suchdaten...")
    loc_title_df = pd.read_parquet(LOC_TITLE_DATA_FILE)
    print(f"   Bereits abgefragt: {len(loc_title_df):,}")
    print(f"   Davon erfolgreich: {(loc_title_df['loc_found'] == True).sum():,}")
else:
    print(f"\n📂 Keine vorhandenen Titel/Autor-Suchdaten gefunden")
    loc_title_df = pd.DataFrame(columns=[
        'vdeh_id', 'query_type', 'title', 'author',
        'loc_found', 'loc_title', 'loc_authors', 'loc_year', 'loc_publisher',
        'loc_isbn', 'loc_issn', 'loc_pages'
    ])

# Identifiziere Kandidaten (nur englischsprachig!)
if ALWAYS_TA_FOR_ALL_WITH_TITLE_AUTHORS:
    title_author_candidates = df_english[
        (df_english['title'].notna()) &
        (df_english['authors_str'].notna()) &
        (df_english['authors_str'] != '')
    ].copy()
else:
    title_author_candidates = df_english[
        (df_english['isbn'].isna()) &
        (df_english['issn'].isna()) &
        (df_english['title'].notna()) &
        (df_english['authors_str'].notna()) &
        (df_english['authors_str'] != '')
    ].copy()

print(f"\n📋 Titel/Autor-Kandidaten (englisch): {len(title_author_candidates):,}")

# Erstelle Query-Liste
title_queries = title_author_candidates[['id', 'title', 'authors_str']].copy()
title_queries.columns = ['vdeh_id', 'title', 'author']
title_queries['query_type'] = 'TITLE_AUTHOR'

# Filtere bereits abgefragte
if len(loc_title_df) > 0:
    already_queried = set(loc_title_df['vdeh_id'])
    new_title_queries = title_queries[~title_queries['vdeh_id'].isin(already_queried)].copy()
    
    print(f"\n🔍 Abgleich mit vorhandenen Daten:")
    print(f"   Bereits vorhanden: {len(title_queries) - len(new_title_queries):,}")
    print(f"   Neu abzufragen: {len(new_title_queries):,}")
else:
    new_title_queries = title_queries
    print(f"\n🔍 Alle {len(new_title_queries):,} Titel/Autor-Queries sind neu")

# Nur abfragen wenn neue Queries vorhanden
if len(new_title_queries) > 0:
    print(f"\n🔄 Starte LoC Titel/Autor-Abfrage für {len(new_title_queries):,} neue Queries...\n")
    
    from tqdm.auto import tqdm
    
    results = []
    stats = {'found': 0, 'not_found': 0}
    query_count = 0
    
    for _, row in tqdm(new_title_queries.iterrows(), total=len(new_title_queries), desc="🔍 LoC Titel/Autor", unit="queries"):
        # API-Abfrage
        loc_result = query_loc_by_title_author(row['title'], row['author'])
        
        # Ergebnis speichern
        result_row = {
            'vdeh_id': row['vdeh_id'],
            'query_type': row['query_type'],
            'title': row['title'],
            'author': row['author'],
            'loc_found': loc_result is not None,
            'loc_title': loc_result.get('title') if loc_result else None,
            'loc_authors': ', '.join(loc_result.get('authors', [])) if loc_result else None,
            'loc_year': loc_result.get('year') if loc_result else None,
            'loc_publisher': loc_result.get('publisher') if loc_result else None,
            'loc_isbn': loc_result.get('isbn') if loc_result else None,
            'loc_issn': loc_result.get('issn') if loc_result else None,
            'loc_pages': loc_result.get('pages') if loc_result else None
        }
        
        results.append(result_row)
        
        if loc_result:
            stats['found'] += 1
        else:
            stats['not_found'] += 1
        
        query_count += 1
        
        # Regelmäßiges Speichern
        if query_count % SAVE_INTERVAL == 0:
            new_results_df = pd.DataFrame(results)
            loc_title_df = pd.concat([loc_title_df, new_results_df], ignore_index=True)
            loc_title_df.to_parquet(LOC_TITLE_DATA_FILE, index=False)
            results = []
            
            current_rate = stats['found'] / query_count * 100
            print(f"💾 Zwischenstand: {query_count}/{len(new_title_queries)} | Erfolgsrate: {current_rate:.1f}%")
        
        # Rate Limiting
        time.sleep(RATE_LIMIT_DELAY)
    
    # Finale Speicherung
    if len(results) > 0:
        new_results_df = pd.DataFrame(results)
        loc_title_df = pd.concat([loc_title_df, new_results_df], ignore_index=True)
        loc_title_df.to_parquet(LOC_TITLE_DATA_FILE, index=False)
    
    print(f"\n💾 LoC Titel/Autor-Daten gespeichert: {LOC_TITLE_DATA_FILE.name}")
    
    print(f"\n📊 === NEUE TITEL/AUTOR-ABFRAGEN ===")
    print(f"   Neue Queries: {len(new_title_queries):,}")
    print(f"   ✅ Gefunden: {stats['found']:,} ({stats['found']/len(new_title_queries)*100:.1f}%)")
    print(f"   ❌ Nicht gefunden: {stats['not_found']:,} ({stats['not_found']/len(new_title_queries)*100:.1f}%)")

else:
    print(f"\n✅ Alle Titel/Autor-Kombinationen bereits abgefragt")

# Gesamtstatistik
print(f"\n📊 === GESAMT TITEL/AUTOR-DATEN ===")
print(f"   Total Records: {len(loc_title_df):,}")
print(f"   Erfolgreich: {(loc_title_df['loc_found'] == True).sum():,}")
print(f"   Nicht gefunden: {(loc_title_df['loc_found'] == False).sum():,}")
if len(loc_title_df) > 0:
    print(f"   📈 Erfolgsrate: {(loc_title_df['loc_found'] == True).sum()/len(loc_title_df)*100:.1f}%")

🔍 === LOC TITEL/AUTOR-SUCHE ===

⚙️  Konfiguration:
   Rate Limit: 10.0s pro Anfrage
   Save Interval: Alle 50 Queries
   Output: loc_title_author_data.parquet
   TA für alle Titel+Autoren: True

📂 Lade vorhandene Titel/Autor-Suchdaten...
   Bereits abgefragt: 4,770
   Davon erfolgreich: 853

📋 Titel/Autor-Kandidaten (englisch): 4,770

🔍 Abgleich mit vorhandenen Daten:
   Bereits vorhanden: 4,770
   Neu abzufragen: 0

✅ Alle Titel/Autor-Kombinationen bereits abgefragt

📊 === GESAMT TITEL/AUTOR-DATEN ===
   Total Records: 4,770
   Erfolgreich: 853
   Nicht gefunden: 3,917
   📈 Erfolgsrate: 17.9%


In [7]:
# 🔗 LOC-DATEN MIT VDEH-DATEN ZUSAMMENFÜHREN
print("🔗 === LOC-DATEN MERGE ===\n")

# Starte mit VDEH-Daten
df_enriched = df_vdeh.copy()

# 1. Merge ISBN/ISSN-basierte LoC-Daten
if len(loc_data_df) > 0:
    cols_to_merge = ['vdeh_id', 'query_type', 'loc_title', 'loc_authors', 'loc_year', 'loc_publisher']
    if 'loc_isbn' in loc_data_df.columns:
        cols_to_merge.extend(['loc_isbn', 'loc_issn', 'loc_pages'])
    
    loc_isbn_issn = loc_data_df[loc_data_df['loc_found'] == True][cols_to_merge].rename(
        columns={'query_type': 'loc_query_method'}
    )
    
    df_enriched = df_enriched.merge(
        loc_isbn_issn,
        left_on='id',
        right_on='vdeh_id',
        how='left',
        suffixes=('', '_dup')
    )
    if 'vdeh_id' in df_enriched.columns:
        df_enriched.drop(columns=['vdeh_id'], inplace=True)
    
    print(f"✅ ISBN/ISSN-basierte LoC-Daten (ID) gemerged")
    print(f"   ID-Matches: {df_enriched['loc_query_method'].notna().sum():,}")

# 2. Merge Titel/Autor-basierte LoC-Daten als separate Variante (_ta)
if len(loc_title_df) > 0:
    cols_to_merge_ta = ['vdeh_id', 'loc_title', 'loc_authors', 'loc_year', 'loc_publisher']
    if 'loc_isbn' in loc_title_df.columns:
        cols_to_merge_ta.extend(['loc_isbn', 'loc_issn', 'loc_pages'])
    
    loc_title_matches = loc_title_df[loc_title_df['loc_found'] == True][cols_to_merge_ta].copy()
    
    rename_map = {
        'loc_title': 'loc_title_ta',
        'loc_authors': 'loc_authors_ta',
        'loc_year': 'loc_year_ta',
        'loc_publisher': 'loc_publisher_ta'
    }
    if 'loc_isbn' in cols_to_merge_ta:
        rename_map.update({
            'loc_isbn': 'loc_isbn_ta',
            'loc_issn': 'loc_issn_ta',
            'loc_pages': 'loc_pages_ta'
        })
    
    loc_title_matches = loc_title_matches.rename(columns=rename_map)
    
    df_enriched = df_enriched.merge(
        loc_title_matches,
        left_on='id',
        right_on='vdeh_id',
        how='left'
    )
    if 'vdeh_id' in df_enriched.columns:
        df_enriched.drop(columns=['vdeh_id'], inplace=True)
    
    print(f"✅ Titel/Autor-basierte LoC-Daten (TA) gemerged")
    print(f"   TA-Matches: {df_enriched[['loc_title_ta','loc_authors_ta','loc_year_ta','loc_publisher_ta']].notna().any(axis=1).sum():,}")

# Zusammenfassung
print(f"\n📊 === MERGE ZUSAMMENFASSUNG ===")
print(f"   Total Records: {len(df_enriched):,}")
print(f"   Mit ID-LoC: {df_enriched['loc_query_method'].notna().sum() if 'loc_query_method' in df_enriched.columns else 0:,}")
print(f"   Mit TA-LoC: {df_enriched[['loc_title_ta','loc_authors_ta','loc_year_ta','loc_publisher_ta']].notna().any(axis=1).sum() if 'loc_title_ta' in df_enriched.columns else 0:,}")


🔗 === LOC-DATEN MERGE ===



✅ ISBN/ISSN-basierte LoC-Daten (ID) gemerged
   ID-Matches: 797


✅ Titel/Autor-basierte LoC-Daten (TA) gemerged
   TA-Matches: 853

📊 === MERGE ZUSAMMENFASSUNG ===
   Total Records: 58,305
   Mit ID-LoC: 797
   Mit TA-LoC: 853


In [8]:
# 🔧 DATENTYP-NORMALISIERUNG
print("🔧 === DATENTYP-NORMALISIERUNG ===\n")

# Konvertiere Jahr-Spalten zu Int64
year_columns = ['year', 'loc_year', 'loc_year_ta']

for col in year_columns:
    if col in df_enriched.columns:
        original_count = df_enriched[col].notna().sum()
        df_enriched[col] = pd.to_numeric(df_enriched[col], errors='coerce').astype('Int64')
        new_count = df_enriched[col].notna().sum()
        
        print(f"   {col}: {original_count:,} → {new_count:,} (Int64)")
        
        if original_count != new_count:
            print(f"      ⚠️  {original_count - new_count:,} Werte konnten nicht konvertiert werden")

print(f"\n✅ Datentypen normalisiert")

🔧 === DATENTYP-NORMALISIERUNG ===

   year: 33,313 → 33,313 (Int64)
   loc_year: 311 → 311 (Int64)
   loc_year_ta: 454 → 454 (Int64)

✅ Datentypen normalisiert


In [9]:
# 💾 DATEN SPEICHERNprint("💾 === DATEN SPEICHERN ===\n")# Output-Pfadeoutput_path = processed_dir / '04b_loc_enriched_data.parquet'metadata_output = processed_dir / '04b_metadata.json'# 1. Parquet speicherndf_enriched.to_parquet(output_path, index=False)print(f"✅ LoC-angereicherte Daten gespeichert: {output_path.name}")print(f"   Records: {len(df_enriched):,}")print(f"   Spalten: {len(df_enriched.columns)}")print(f"   Größe: {output_path.stat().st_size / 1024**2:.1f} MB")# 2. Metadaten erstellenmetadata = {    'step': '04b_loc_enrichment',    'input_file': '03_language_detected_data.parquet',    'output_file': '04b_loc_enriched_data.parquet',    'timestamp': pd.Timestamp.now().isoformat(),    'record_count': len(df_enriched),    'english_records': len(df_english),    'columns': list(df_enriched.columns),        'loc_queries': {        'isbn_issn': {            'total_queries': len(loc_data_df) if len(loc_data_df) > 0 else 0,            'successful': int((loc_data_df['loc_found'] == True).sum()) if len(loc_data_df) > 0 else 0,            'failed': int((loc_data_df['loc_found'] == False).sum()) if len(loc_data_df) > 0 else 0        },        'title_author': {            'total_queries': len(loc_title_df) if len(loc_title_df) > 0 else 0,            'successful': int((loc_title_df['loc_found'] == True).sum()) if len(loc_title_df) > 0 else 0,            'failed': int((loc_title_df['loc_found'] == False).sum()) if len(loc_title_df) > 0 else 0        }    },        'loc_variants': {        'id_available': int(df_enriched['loc_query_method'].notna().sum()) if 'loc_query_method' in df_enriched.columns else 0,        'ta_available': int(df_enriched[['loc_title_ta','loc_authors_ta','loc_year_ta','loc_publisher_ta']].notna().any(axis=1).sum()) if 'loc_title_ta' in df_enriched.columns else 0    }}# Metadaten speichernwith open(metadata_output, 'w', encoding='utf-8') as f:    json.dump(metadata, f, indent=2, ensure_ascii=False)print(f"\n✅ Metadaten gespeichert: {metadata_output.name}")# 3. Zusammenfassungprint(f"\n📊 === LOC ENRICHMENT ABGESCHLOSSEN ===")print(f"   Input: {len(df_vdeh):,} VDEH Records")print(f"   Englischsprachig: {len(df_english):,}")print(f"   Output: {len(df_enriched):,} Records mit LoC-Daten")print(f"   ID-Variante verfügbar: {metadata['loc_variants']['id_available']:,}")print(f"   TA-Variante verfügbar: {metadata['loc_variants']['ta_available']:,}")print(f"\n➡️  Nächster Schritt: Integration mit DNB-Daten in 05_vdeh_data_fusion.ipynb")print(f"\n🎉 LoC Enrichment erfolgreich abgeschlossen!")